In [1]:
import csv
from config import api
import pandas as pd
import requests

In [2]:
def problem_dd():
    problem_list = []
    with open('static/data/problem.csv') as problem_file:
        problem = csv.reader(problem_file, delimiter=',')
#       skip header row
        next(problem, None)
        for row in problem:
            problem_list.append({'prob_id':row[0], 'prob_desc': row[1]})
        return problem_list

problem_list = problem_dd()

In [3]:
print(problem_list)

[{'prob_id': '1', 'prob_desc': 'High Cholestrol'}, {'prob_id': '2', 'prob_desc': 'High Blood Pressure'}, {'prob_id': '3', 'prob_desc': 'Gout'}, {'prob_id': '5', 'prob_desc': 'Vitamin D Deficiency'}, {'prob_id': '6', 'prob_desc': 'Breast Cancer'}, {'prob_id': '9', 'prob_desc': 'Prostrate Cancer'}, {'prob_id': '12', 'prob_desc': 'Stomach Cancer'}, {'prob_id': '15', 'prob_desc': 'Rhematoid Arthiritis'}, {'prob_id': '16', 'prob_desc': 'Diabetes Type 2'}, {'prob_id': '18', 'prob_desc': 'Depression'}, {'prob_id': '27', 'prob_desc': 'Celiac Disease'}, {'prob_id': '28', 'prob_desc': "Crohn's Disease"}, {'prob_id': '34', 'prob_desc': 'Ulcers'}]


In [4]:
base_url="https://nutridigm-api-dev.azurewebsites.net"
consume_url="/api/v1/nutridigm/topitemstoconsume?"
avoid_url="/api/v1/nutridigm/topitemstoavoid?"


# Create Data Frame
problem_df = pd.DataFrame()

# Iterrate over the list problem_list
x = 1

for problem in problem_list:
    problemId=problem['prob_id']
    query_consume=base_url + consume_url + f'subscriptionId={api}&problemId={problemId}&api_key={api}'
    query_avoid=base_url + avoid_url + f'subscriptionId={api}&problemId={problemId}&api_key={api}'
    
    # Get the food data from nudrigim.org
    c_food = requests.get(query_consume).json()
    a_food = requests.get(query_avoid).json()
    
    if x!=1:
        for c in c_food:
            consume_food = c
    
        for a in a_food:
            avoid_food = a
    
    
            problem_df.loc[x, "ID"]= problem['prob_id']
            problem_df.loc[x, "Health Problem"]=problem['prob_desc']
            problem_df.loc[x, "Consume"] = consume_food
            problem_df.loc[x, "Avoid"] = avoid_food
    
    else:
            problem_df.loc[x, "ID"]= problem['prob_id']
            problem_df.loc[x, "Health Problem"]=problem['prob_desc']
            problem_df.loc[x, "Consume"] = c_food
            problem_df.loc[x, "Avoid"] = a_food

    x +=1
problem_df

,ID,Health Problem,Consume,Avoid
1,1,High Cholestrol,"EXERCISE; Barley, Fiber/Psyllium; Fish; Beans/...",Animal Fat & Lard; Margarine & Cheese; Sweets;...
2,2,High Blood Pressure,Green Leafy Vegetables & Sun-dried Tomatoes; F...,2+ alcoholic drinks/day; Sweets & Chocolate; S...
3,3,Gout,Cherries & Berries; Water & Juices; Cereals; R...,Fish & Fish Roe; Beer & 2+ Alcoholic drinks/da...
4,5,Vitamin D Deficiency,Catch some sun; Fish; Cod Liver Fish Oil; Egg ...,Obesity/Excess Weight; Processed & Cured Meats...
5,6,Breast Cancer,"Broccoli, Cruciferous and Green Leafy vegetabl...",Wheat/Whole-wheat; Sweets & Sugar; Pesticide-l...
6,9,Prostrate Cancer,"Berries & other fruits; Garlic, Onions & Tomat...","Wheat, Bread & Cereals; Sweets & Sugar; Lunche..."
7,12,Stomach Cancer,Garlic & Peas; Green tea; Cabbage Family & Gre...,"Wheat, Bread & Cereals; Sweets & Sugar; Game &..."
8,15,Rhematoid Arthiritis,"Ginger; Beans/Legumes; Berries, Plum, Pomegran...",Sweets & Sugar; Nuts & Seeds; Luncheon & Cured...
9,16,Diabetes Type 2,"Exercise; Broccoli, Carrots, Potatoes & Spinac...",2+ alcoholic drinks/day; Sweets & Refined suga...
10,18,Depression,EXERCISE; Fish & Shellfish; Liver; Egg yolk; G...,"2+ alcohol drinks/day; Sweets, Sugar & Corn Sy..."


In [5]:
# Export the data into a json
problem_df.to_json("static/data/health_problem.json", orient='records')

In [6]:
# Export the data into a csx

problem_df.set_index("ID", inplace=True)
problem_df.to_csv("static/data/health_problem.csv")